In [ ]:
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm

In [ ]:
# Cell for starting training
# from start import init_train
#
# Trainer = init_train("configs/StyleGAN_256.json", True)

In [ ]:
from start import init_train
from utils.images import SaveImages

# Loading models and the latest weights without loading the dataset
Trainer = init_train("configs/StyleGAN_256.json", load_dataset=False)

# Save 1000 randomly generated images to the img/256_2 folder
SaveImages(Trainer, dir='img/256_2', cnt=1000)

In [ ]:
# !tar -cvf img.tar img/256_2

In [ ]:
from utils.images import TensorToImage

plot_width, plot_height = 6, 20
fig = plt.figure(figsize=(4 * plot_width, 4 * plot_height))

z = torch.randn((plot_width * plot_height, 256), device=Trainer.device)
for sample_n in range(1, plot_width * plot_height + 1):
    ax = fig.add_subplot(plot_height, plot_width, sample_n)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.imshow(TensorToImage(Trainer.G(z[sample_n - 1])[0].detach().cpu(), 0.5, 0.4))
    plt.title(sample_n - 1)

In [ ]:
from utils.video import GenerateVideo

def FromToVideo(z_1, z_2, frames=200):
    samples = []
    z_1 = z_1.to(Trainer.device)
    z_2 = z_2.to(Trainer.device)
    delta = ((z_2 - z_1)/(frames - 1)).to(Trainer.device)
    for i in range(frames):
        samples.append(Trainer.G(z_1 + i*delta).detach().cpu()[0])
    return samples

In [ ]:
samples = []
images = [0, 44, 46, 115]
for to in tqdm(range(1, len(images))):
    a = images[to - 1]
    b = images[to]
    samples_to = FromToVideo(z[a], z[b], 350)
    samples = [*samples, *samples_to]
samples_to = FromToVideo(z[images[-1]], z[images[0]], 350)
samples = [*samples, *samples_to]

GenerateVideo(samples, 1000/60, 0.5, 0.4)